In [1]:
import pandas as pd
import numpy as np
import scipy
from nltk.corpus import stopwords
import re
import os

In [2]:
## Loading data - an excell file :

data1 = pd.ExcelFile('Tranzact2.xlsx')
tranzact_data1 = data1.parse(0)

lamba = len(tranzact_data1)

tranzact_data2 = tranzact_data1.iloc[0:lamba, 0:3] ## Remove unnecessary columns

tranzact_data2['Item'] = tranzact_data2['Item'].map(lambda x: x if type(x)!=str else x.lower()) ## make Items in lower case


tranzact_data3 = tranzact_data2.dropna() ## remove rows with cell value none

tranzact_data3 = tranzact_data3.reset_index(drop=True)

## Remove items sold by a company to its subsidiary :

drop_list = []

for i in range(len(tranzact_data3)):
    
    if (tranzact_data3['Supplier'][i].split())[0] == 'CESARE'and (tranzact_data3['Company'][i].split())[0] == 'CESARE':
        
        drop_list.append(i)
        
        tranzact_data = tranzact_data3.drop(drop_list)
                
tranzact_data = tranzact_data.reset_index(drop=True)

print(len(tranzact_data))
tranzact_data.head()       

24348


,Company,Supplier,Item
0,M/S Plastech,Pukhraj Additives LLP,zincoplast zinc
1,M/S Plastech,Pukhraj Additives LLP,zincoplast pyn
2,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."
3,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."
4,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."


In [3]:
## Cleaning the Item Description to create a feature matrix :

pd.options.mode.chained_assignment = None

tranzact_data['features'] = tranzact_data['Item'].str.replace(r'\b\d+\b','')  ## remove integers

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\W',' ')   ## remove puntuations

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\w\b','') ## remove stand alone single letters

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\s+',' ') ## remove gaps between words to singe gap

## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")
words.remove('for')
words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')
words.append('max')
words.append('min')
#words.append('machined')
words.append('top')
words.append('dia')

tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())


## Removing duplicate words from individual features

tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
                                                                                key=x.split().index))]).lower())
print(len(tranzact_data))

## Removing only single word features :

tranzact_data = tranzact_data[tranzact_data['features'].str.contains(' ')]
tranzact_data = tranzact_data.reset_index(drop=True)

print(len(tranzact_data))

24348
22872


In [4]:
for i in range(len(tranzact_data)):
    if tranzact_data['features'][i] == '':
        
        print(i)
        print('===')
        tranzact_data['Supplier'][i] = np.nan
    
tranzact_data = tranzact_data.dropna()
tranzact_data = tranzact_data.reset_index(drop=True)
print(len(tranzact_data))
tranzact_data.tail()

22872


,Company,Supplier,Item,features
22867,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'r',ferrul number
22868,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'y',ferrul number
22869,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'b',ferrul number
22870,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'n',ferrul number
22871,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long


In [5]:
## Removing duplicate features + Supplier combinations :

tranzact_data['find_duplicate'] = tranzact_data['Supplier'].map(str) + tranzact_data['features']
tranzact_data.drop_duplicates(subset='find_duplicate', keep = 'first', inplace = True)
tranzact_data = tranzact_data.reset_index(drop=True)
print(len(tranzact_data))
tranzact_data.tail()

16383


,Company,Supplier,Item,features,find_duplicate
16378,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg11,pvc gland pg11,M/S. S. K. Enterprisepvc gland pg11
16379,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg13.5,pvc gland pg13,M/S. S. K. Enterprisepvc gland pg13
16380,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland m25x1.5,pvc gland m25x1,M/S. S. K. Enterprisepvc gland m25x1
16381,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 1mm '1',ferrul number,M/S. S. K. Enterpriseferrul number
16382,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long,Anant Tradershard copper pipe gauge long


In [6]:
## Removing unnecessary long features ( feature words > 25):

tranzact_data['number_of_words'] = tranzact_data.features.apply(lambda x: len(x.split()))

list_to_drop = []

for i in range(len(tranzact_data['number_of_words'])):
    
    if tranzact_data['number_of_words'][i] > 50 :
        
        list_to_drop.append(i)

tranzact_data_final = tranzact_data.drop(list_to_drop)
tranzact_data_final = tranzact_data_final.reset_index(drop=True)

tranzact_data_final.tail()

,Company,Supplier,Item,features,find_duplicate,number_of_words
16369,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg11,pvc gland pg11,M/S. S. K. Enterprisepvc gland pg11,3
16370,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg13.5,pvc gland pg13,M/S. S. K. Enterprisepvc gland pg13,3
16371,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland m25x1.5,pvc gland m25x1,M/S. S. K. Enterprisepvc gland m25x1,3
16372,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 1mm '1',ferrul number,M/S. S. K. Enterpriseferrul number,2
16373,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long,Anant Tradershard copper pipe gauge long,5


In [5]:
## To check no. of Suppliers & no. of unique featuresin the data base :

# print('Supplier = ', len(tranzact_data.Supplier.value_counts()))
# print('Unique Features = ', len(tranzact_data.features.value_counts()))
# print('Tranzact_data length = ', len(tranzact_data))
# tranzact_data.tail()

In [7]:
## Above indicates that there are 2337 suppliers supplying 28921 items(as per no. of rows)
## above indicates that there are 16728 unique features out of 28921 items(as per rows)

In [8]:
## Let's reduce no. of Suppliers : criterian - apperaing at least 10 times in the data frame

# tranzact_data_short = tranzact_data.groupby("Supplier").filter(lambda x: len(x) > 10)

# print(len(tranzact_data_short))
# print(len(tranzact_data_short.Supplier.value_counts()))

In [9]:
## Above indicates that there are now 391 suppliers supplying 23649 items(as per no. of rows)

In [60]:
## Removing duplicate features + Supplier combinations :

for i in range (len(tranzact_data)):
    
    idx_feature = []
    idx_Supplier = []
    idx_common = []

    idx_feature = tranzact_data.index[tranzact_data['features'] == tranzact_data['features'][i]].tolist()
    idx_Supplier = tranzact_data.index[tranzact_data['Supplier'] == tranzact_data['Supplier'][i]].tolist()
    idx_common = list(set(idx_feature).intersection(idx_Supplier))
    
    if len(idx_common) > 1:
        
        tranzact_data['Supplier'][idx_common[1:]] = np.nan
        
tranzact_data_final = tranzact_data.dropna()
tranzact_data_final = tranzact_data_final.reset_index(drop=True)
print(len(tranzact_data_final))

## Removing unnecessary long features ( feature words > 25):



18841


In [8]:
## To cross check no. of Suppliers & no. of unique features in the data base :

# print('Supplier = ', len(tranzact_data_final.Supplier.value_counts()))
# print('Unique Features = ', len(tranzact_data_final.features.value_counts()))
# print('Total no. of entrys = ', len(tranzact_data_final))
# tranzact_data_final.tail()

In [8]:
# Creating seacrh engine file  and digitising original features :

print(len(tranzact_data_final))

from sklearn.feature_extraction.text import TfidfVectorizer

col = ['Supplier', 'features']
tranzact_data_search = tranzact_data_final[col]


tranzact_data_search['Supplier_id'] = tranzact_data_search['Supplier'].factorize()[0]  ## Indexing

supplier_id_tranzact_data_search = tranzact_data_search[['Supplier', 'Supplier_id']].drop_duplicates(). \
                                                        sort_values('Supplier_id')

## Create two dictionaries :

supplier_to_id = dict(supplier_id_tranzact_data_search.values)
id_to_supplier = dict(supplier_id_tranzact_data_search[['Supplier_id', 'Supplier']].values)

## Digitising :

print(len(tranzact_data_search['features']))
print(len(tranzact_data_search['features'][0]))
print(tranzact_data_search['features'][0])

my_vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', ngram_range=(1, 2))
digital_features = my_vectorizer.fit(tranzact_data_search['features'])
my_vector = digital_features.transform(tranzact_data_search['features']).toarray()
print(my_vector.shape)
print(tranzact_data_search['features'][12490])
tranzact_data_search.tail()

16374
16374
15
zincoplast zinc
(16374, 46457)
dowel pin uni1707 d12 l60


,Supplier,features,Supplier_id
16369,M/S. S. K. Enterprise,pvc gland pg11,309
16370,M/S. S. K. Enterprise,pvc gland pg13,309
16371,M/S. S. K. Enterprise,pvc gland m25x1,309
16372,M/S. S. K. Enterprise,ferrul number,309
16373,Anant Traders,hard copper pipe gauge long,2056


In [15]:
## Input Text data :

text_data = ['pin uni1707']

In [ ]:
## From Teaxt data to text feature :

text_data_lower_case = [x.lower() for x in text_data]

text_data1 = pd.Series(text_data_lower_case)

pd.options.mode.chained_assignment = None

text_data_feature = text_data1.str.replace(r'\b\d+\b','')  ## remove integers

text_data_feature = text_data_feature.str.replace(r'\W',' ')   ## remove puntuations

text_data_feature = text_data_feature.str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

text_data_feature = text_data_feature.str.replace(r'\b\w\b','') ## remove stand alone single letters

text_data_feature = text_data_feature.str.replace(r'\s+',' ') ## remove gaps between words to singe gap


## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")
words.remove('for')
words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')
words.append('max')
#words.append('machined')
words.append('top')
words.append('dia')

text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())

text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
                                                                                key=x.split().index))]).lower())

print(text_data_feature)
## Cosine Similarity :

def cos_cdist(matrix, vector):
    v = vector.reshape(1, -1)
    return scipy.spatial.distance.cdist(matrix, v, 'cosine').reshape(-1)

## Creating test vector from Text features :

merge_text_feature = []
merge_text_feature_digit = []
merge_digit_vector = []
test_digit_vector = []

merge_text_feature = tranzact_data_search['features'].append(pd.Series(text_data_feature))

merge_text_feature = merge_text_feature.reset_index(drop=True)

merge_text_feature_digit = my_vectorizer.fit(merge_text_feature)

# encode document
merge_digit_vector = merge_text_feature_digit.transform(merge_text_feature).toarray()

test_digit_vector = merge_digit_vector[len(merge_digit_vector)-1].reshape((merge_digit_vector.shape)[1],)


## Finding cosine similarity :

c_d = []

c_d = np.round((1 - cos_cdist(my_vector, test_digit_vector)), 3)
print(type(c_d))
print(c_d.shape)

0    pin uni1707
dtype: object
(16375, 46457)
<class 'numpy.ndarray'>


In [17]:
result = []
result_frame = []
max_c_s_index = []
max_c_s_index = np.argsort(c_d)[-50:][::-1]

for i in max_c_s_index:
    
    result.append(np.array([c_d[i], tranzact_data_final['Item'][i], tranzact_data_search['features'][i], \
                            tranzact_data_search['Supplier'][i]]))

result_frame = pd.DataFrame(result, columns=['c_index', 'Item', 'features', 'Supplier'])

result_frame.c_index = result_frame.c_index.astype('float64')

result_frame.drop_duplicates(subset='Supplier', keep = 'first', inplace = True)
result_frame = result_frame.reset_index(drop=True)

kount = 0

for i in range(len(result_frame['Supplier'])):
    if result_frame['c_index'][i] >= 0.10:
        print(result_frame['Supplier'][i])
        kount = kount+1
        if kount == 5 :
            break
result_frame.head(10)

DAMODAR ENTERPRISES
HAFSA INDUSTRIES
Nisham Enterprises
R.P Engineering Works
M. & D. Engineers Pvt. Ltd.


,c_index,Item,features,Supplier
0,0.669,pin uni1707 d18x20,pin uni1707 d18x20,DAMODAR ENTERPRISES
1,0.521,dowel pin uni1707 d12 l60,dowel pin uni1707 d12 l60,HAFSA INDUSTRIES
2,0.179,machined pin - 60 (d) x 1050 (l),machined pin,Nisham Enterprises
3,0.179,machined pin - 25 x 180 mm,machined pin,R.P Engineering Works
4,0.164,ms pin,ms pin,M. & D. Engineers Pvt. Ltd.
5,0.163,dowel pin 4 x 30mm,dowel pin,Alpha Enterprises
6,0.163,dowel pin m 10 x 100 mm,dowel pin,Patel Fasteners
7,0.162,spring pin,spring pin,Dave Spring & Engg. Co.
8,0.162,6 pin plug,pin plug,Anumita Electricals
9,0.159,pin -tpw,pin tpw,Maa Sitola Engineering Works


In [24]:
c_d_sort = -np.sort(-c_d)

In [27]:
print(c_d_sort[0])

0.669
